Introducción a Bases de Datos MySQL con Python
===

Usando las técnicas a continuación, podrá integrar de manera eficiente una base de datos MySQL con una aplicación de Python. Tendrá la posibilidad de construir una pequeña base de datos MySQL para un sistema de clasificación de películas y aprenderá a conectar, consultar y modificar sus datos.

El primer paso es realizar la instalación MySQL en su entorno local:
- [Installing and Upgrading MySQL](https://dev.mysql.com/doc/refman/5.7/en/installing.html)
- [MySQL Community Downloads](https://dev.mysql.com/downloads/installer/)

Como prerequisito final, debe instalar el [conector de MySQL para Python](https://github.com/mysql/mysql-connector-python):
- Ejecute el comando ```pip install mysql-connector-python```en consola (bash, cmd, etc.)
- Pruebe la instalación, ingrese por consola a ```python3``` y ejecute ```import mysql.connector```

## Contenido
>- [Estableciendo una conexión](#Estableciendo-una-conexión)
>- [Creación, modificación y eliminación de una tabla](#Creación,-modificación-y-eliminación-de-una-tabla)
>- [Insertando registros en una tabla](#Insertando-registros-en-una-tabla)
>- [Consultando registros de la base de datos](#Consultando-registros-de-la-base-de-datos)
>- [Filtrando resultados usando ```where```](#Filtrando-resultados-usando-where)

>- [Encapsulación en Python](#Encapsulación-en-Python)
>- [Polimorfismo en Python](#Polimorfismo-en-Python)
>- [Variables de Clase o Estáticas](#Variables-de-Clase-o-Estáticas)
>- [Método de Clase vs Método Estático](#Método-de-Clase-vs-Método-Estático)

In [7]:
from getpass import getpass
from mysql.connector import connect, Error

## Estableciendo una conexión

In [4]:
# Establecemos una conexión a la base de datos MySQL

try:
    with connect(
        host="localhost",
        user=input("Enter username: "),
        password=getpass("Enter password: "),
    ) as connection:
        print(connection)
except Error as e:
    print(e)

Enter username: root
Enter password: ········


In [5]:
# Creamos una nueva base de datos en MySQL

try:
    with connect(
        host="localhost",
        user=input("Enter username: "),
        password=getpass("Enter password: "),
    ) as connection:
        create_db_query = "CREATE DATABASE online_movie_rating"
        with connection.cursor() as cursor:
            cursor.execute(create_db_query)
except Error as e:
    print(e)

Enter username: root
Enter password: ········


In [23]:
# Verificamos que la base de datos haya sido creada

try:
    with connect(
        host="localhost",
        user=input("Enter username: "),
        password=getpass("Enter password: "),
    ) as connection:
        show_db_query = "SHOW DATABASES"
        with connection.cursor() as cursor:
            cursor.execute(show_db_query)
            print('online_movie_rating', 
                  'exists :)' if 'online_movie_rating' in [db[0] for db in cursor] \
                  else 'not exists :(')
except Error as e:
    print(e)

Enter username: root
Enter password: ········
online_movie_rating exists :)


In [24]:
#  Creamos una conexión a la nueva base de datos

try:
    with connect(
        host="localhost",
        user=input("Enter username: "),
        password=getpass("Enter password: "),
        database="online_movie_rating",
    ) as connection:
        print(connection)
except Error as e:
    print(e)

Enter username: root
Enter password: ········


## Creación, modificación y eliminación de una tabla

![schema-movies.webp](./images/schema-movies.webp)

In [25]:
# Creamos una variable con la conexión

try:
    connection = connect(
        host="localhost",
        user=input("Enter username: "),
        password=getpass("Enter password: "),
        database="online_movie_rating",
    )
    print(connection)
except Error as e:
    print(e)

Enter username: root
Enter password: ········


In [26]:
# Creamos la tabla "movies"

create_movies_table_query = """
    CREATE TABLE movies(
        id INT AUTO_INCREMENT PRIMARY KEY,
        title VARCHAR(100),
        release_year YEAR(4),
        genre VARCHAR(100),
        collection_in_mil INT
    )
"""
with connection.cursor() as cursor:
    cursor.execute(create_movies_table_query)
    connection.commit()

In [27]:
# Creamos la tabla reviewers

create_reviewers_table_query = """
    CREATE TABLE reviewers (
        id INT AUTO_INCREMENT PRIMARY KEY,
        first_name VARCHAR(100),
        last_name VARCHAR(100)
    )
"""
with connection.cursor() as cursor:
    cursor.execute(create_reviewers_table_query)
    connection.commit()

In [28]:
# Creamos la tabla ratings

create_ratings_table_query = """
    CREATE TABLE ratings (
        movie_id INT,
        reviewer_id INT,
        rating DECIMAL(2,1),
        FOREIGN KEY(movie_id) REFERENCES movies(id),
        FOREIGN KEY(reviewer_id) REFERENCES reviewers(id),
        PRIMARY KEY(movie_id, reviewer_id)
    )
"""
with connection.cursor() as cursor:
    cursor.execute(create_ratings_table_query)
    connection.commit()

In [29]:
# Creamos todas las tablas en secuencia

with connection.cursor() as cursor:
    try:
        cursor.execute(create_movies_table_query)
        cursor.execute(create_reviewers_table_query)
        cursor.execute(create_ratings_table_query)
        connection.commit()
    except:
        connection.rollback()

In [30]:
# Vemos estructura de tabla con DESCRIBE

show_table_query = "DESCRIBE movies"
with connection.cursor() as cursor:
    cursor.execute(show_table_query)
    # Fetch rows from last executed query
    result = cursor.fetchall()
    for row in result:
        print(row)

('id', b'int', 'NO', 'PRI', None, 'auto_increment')
('title', b'varchar(100)', 'YES', '', None, '')
('release_year', b'year', 'YES', '', None, '')
('genre', b'varchar(100)', 'YES', '', None, '')
('collection_in_mil', b'int', 'YES', '', None, '')


In [31]:
# Modificamos tipo de dato para un campo con ALTER

alter_table_query = """
    ALTER TABLE movies
        MODIFY COLUMN collection_in_mil DECIMAL(4,1)
"""
show_table_query = "DESCRIBE movies"
with connection.cursor() as cursor:
    cursor.execute(alter_table_query)
    cursor.execute(show_table_query)
    # Fetch rows from last executed query
    result = cursor.fetchall()
    print("Movie Table Schema after alteration:")
    for row in result:
        print(row)

Movie Table Schema after alteration:
('id', b'int', 'NO', 'PRI', None, 'auto_increment')
('title', b'varchar(100)', 'YES', '', None, '')
('release_year', b'year', 'YES', '', None, '')
('genre', b'varchar(100)', 'YES', '', None, '')
('collection_in_mil', b'decimal(4,1)', 'YES', '', None, '')


In [32]:
drop_table_query = "DROP TABLE ratings"
with connection.cursor() as cursor:
    cursor.execute(drop_table_query)

In [33]:
show_table_query = "DESCRIBE ratings"
with connection.cursor() as cursor:
    cursor.execute(show_table_query)
    # Fetch rows from last executed query
    result = cursor.fetchall()
    for row in result:
        print(row)

ProgrammingError: 1146 (42S02): Table 'online_movie_rating.ratings' doesn't exist

In [38]:
# Creamos nuevamente la tabla ratings para el ejercicio

with connection.cursor() as cursor:
    cursor.execute(create_ratings_table_query)
    connection.commit()

## Insertando registros en una tabla

In [35]:
# Insertando registros en movies usando execute()

insert_movies_query = """
    INSERT INTO movies (title, release_year, genre, collection_in_mil)
    VALUES
        ("Forrest Gump", 1994, "Drama", 330.2),
        ("3 Idiots", 2009, "Drama", 2.4),
        ("Eternal Sunshine of the Spotless Mind", 2004, "Drama", 34.5),
        ("Good Will Hunting", 1997, "Drama", 138.1),
        ("Skyfall", 2012, "Action", 304.6),
        ("Gladiator", 2000, "Action", 188.7),
        ("Black", 2005, "Drama", 3.0),
        ("Titanic", 1997, "Romance", 659.2),
        ("The Shawshank Redemption", 1994, "Drama",28.4),
        ("Udaan", 2010, "Drama", 1.5),
        ("Home Alone", 1990, "Comedy", 286.9),
        ("Casablanca", 1942, "Romance", 1.0),
        ("Avengers: Endgame", 2019, "Action", 858.8),
        ("Night of the Living Dead", 1968, "Horror", 2.5),
        ("The Godfather", 1972, "Crime", 135.6),
        ("Haider", 2014, "Action", 4.2),
        ("Inception", 2010, "Adventure", 293.7),
        ("Evil", 2003, "Horror", 1.3),
        ("Toy Story 4", 2019, "Animation", 434.9),
        ("Air Force One", 1997, "Drama", 138.1),
        ("The Dark Knight", 2008, "Action",535.4),
        ("Bhaag Milkha Bhaag", 2013, "Sport", 4.1),
        ("The Lion King", 1994, "Animation", 423.6),
        ("Pulp Fiction", 1994, "Crime", 108.8),
        ("Kai Po Che", 2013, "Sport", 6.0),
        ("Beasts of No Nation", 2015, "War", 1.4),
        ("Andadhun", 2018, "Thriller", 2.9),
        ("The Silence of the Lambs", 1991, "Crime", 68.2),
        ("Deadpool", 2016, "Action", 363.6),
        ("Drishyam", 2015, "Mystery", 3.0)
"""
with connection.cursor() as cursor:
    try:
        cursor.execute(insert_movies_query)
        connection.commit()
    except:
        connection.rollback()

In [40]:
# Insertando registros en reviewers usando executemany()

insert_reviewers_query = """
    INSERT INTO reviewers (first_name, last_name)
        VALUES ( %s, %s )
"""
reviewers_records = [
    ("Chaitanya", "Baweja"),
    ("Mary", "Cooper"),
    ("John", "Wayne"),
    ("Thomas", "Stoneman"),
    ("Penny", "Hofstadter"),
    ("Mitchell", "Marsh"),
    ("Wyatt", "Skaggs"),
    ("Andre", "Veiga"),
    ("Sheldon", "Cooper"),
    ("Kimbra", "Masters"),
    ("Kat", "Dennings"),
    ("Bruce", "Wayne"),
    ("Domingo", "Cortes"),
    ("Rajesh", "Koothrappali"),
    ("Ben", "Glocker"),
    ("Mahinder", "Dhoni"),
    ("Akbar", "Khan"),
    ("Howard", "Wolowitz"),
    ("Pinkie", "Petit"),
    ("Gurkaran", "Singh"),
    ("Amy", "Farah Fowler"),
    ("Marlon", "Crafford"),
]
with connection.cursor() as cursor:
    try:
        cursor.executemany(insert_reviewers_query, reviewers_records)
        connection.commit()
    except:
        connection.rollback()

In [39]:
# Insertando registros en ratings usando executemany()

insert_ratings_query = """
    INSERT INTO ratings (rating, movie_id, reviewer_id)
        VALUES ( %s, %s, %s)
"""
ratings_records = [
    (6.4, 17, 5), (5.6, 19, 1), (6.3, 22, 14), (5.1, 21, 17),
    (5.0, 5, 5), (6.5, 21, 5), (8.5, 30, 13), (9.7, 6, 4),
    (8.5, 24, 12), (9.9, 14, 9), (8.7, 26, 14), (9.9, 6, 10),
    (5.1, 30, 6), (5.4, 18, 16), (6.2, 6, 20), (7.3, 21, 19),
    (8.1, 17, 18), (5.0, 7, 2), (9.8, 23, 3), (8.0, 22, 9),
    (8.5, 11, 13), (5.0, 5, 11), (5.7, 8, 2), (7.6, 25, 19),
    (5.2, 18, 15), (9.7, 13, 3), (5.8, 18, 8), (5.8, 30, 15),
    (8.4, 21, 18), (6.2, 23, 16), (7.0, 10, 18), (9.5, 30, 20),
    (8.9, 3, 19), (6.4, 12, 2), (7.8, 12, 22), (9.9, 15, 13),
    (7.5, 20, 17), (9.0, 25, 6), (8.5, 23, 2), (5.3, 30, 17),
    (6.4, 5, 10), (8.1, 5, 21), (5.7, 22, 1), (6.3, 28, 4),
    (9.8, 13, 1)
]
with connection.cursor() as cursor:
    try:
        cursor.executemany(insert_ratings_query, ratings_records)
        connection.commit()
    except:
        connection.rollback()

## Consultando registros de la base de datos

In [41]:
# Consulta de peliculas usando SELECT

select_movies_query = "SELECT * FROM movies LIMIT 5"
with connection.cursor() as cursor:
    cursor.execute(select_movies_query)
    result = cursor.fetchall()
    for row in result:
        print(row)

(1, 'Forrest Gump', 1994, 'Drama', Decimal('330.2'))
(2, '3 Idiots', 2009, 'Drama', Decimal('2.4'))
(3, 'Eternal Sunshine of the Spotless Mind', 2004, 'Drama', Decimal('34.5'))
(4, 'Good Will Hunting', 1997, 'Drama', Decimal('138.1'))
(5, 'Skyfall', 2012, 'Action', Decimal('304.6'))


In [43]:
# Consulta de peliculas usando SELECT, agrega offset al limit

select_movies_query = "SELECT * FROM movies LIMIT 2,5"
with connection.cursor() as cursor:
    cursor.execute(select_movies_query)
    result = cursor.fetchall()
    for row in result:
        print(row)

(3, 'Eternal Sunshine of the Spotless Mind', 2004, 'Drama', Decimal('34.5'))
(4, 'Good Will Hunting', 1997, 'Drama', Decimal('138.1'))
(5, 'Skyfall', 2012, 'Action', Decimal('304.6'))
(6, 'Gladiator', 2000, 'Action', Decimal('188.7'))
(7, 'Black', 2005, 'Drama', Decimal('3.0'))


In [42]:
# Consulta de peliculas usando SELECT, retorna columnas especificas

select_movies_query = "SELECT title, release_year FROM movies LIMIT 5"
with connection.cursor() as cursor:
    cursor.execute(select_movies_query)
    for row in cursor.fetchall():
        print(row)

('Forrest Gump', 1994)
('3 Idiots', 2009)
('Eternal Sunshine of the Spotless Mind', 2004)
('Good Will Hunting', 1997)
('Skyfall', 2012)


**Ejercicio:** Realice una consulta de películas desde la posición 5 y extraiga las siguientes 10. Muestre únicamente el nombre y género de la película.

## Filtrando resultados usando ```where```

In [44]:
# Consulta las peliculas con recaudación en taquilla > 300 millones

select_movies_query = """
    SELECT title, collection_in_mil
    FROM movies
    WHERE collection_in_mil > 300
    ORDER BY collection_in_mil DESC
"""
with connection.cursor() as cursor:
    cursor.execute(select_movies_query)
    for movie in cursor.fetchall():
        print(movie)

('Avengers: Endgame', Decimal('858.8'))
('Avengers: Endgame', Decimal('858.8'))
('Titanic', Decimal('659.2'))
('Titanic', Decimal('659.2'))
('The Dark Knight', Decimal('535.4'))
('The Dark Knight', Decimal('535.4'))
('Toy Story 4', Decimal('434.9'))
('Toy Story 4', Decimal('434.9'))
('The Lion King', Decimal('423.6'))
('The Lion King', Decimal('423.6'))
('Deadpool', Decimal('363.6'))
('Deadpool', Decimal('363.6'))
('Forrest Gump', Decimal('330.2'))
('Forrest Gump', Decimal('330.2'))
('Skyfall', Decimal('304.6'))
('Skyfall', Decimal('304.6'))


In [45]:
# Consulta peliculas y retorna concatenación de titulo y año

select_movies_query = """
    SELECT CONCAT(title, " (", release_year, ")"),
          collection_in_mil
    FROM movies
    ORDER BY collection_in_mil DESC
    LIMIT 5
"""
with connection.cursor() as cursor:
    cursor.execute(select_movies_query)
    for movie in cursor.fetchall():
        print(movie)

('Avengers: Endgame (2019)', Decimal('858.8'))
('Avengers: Endgame (2019)', Decimal('858.8'))
('Titanic (1997)', Decimal('659.2'))
('Titanic (1997)', Decimal('659.2'))
('The Dark Knight (2008)', Decimal('535.4'))


In [48]:
# Consulta peliculas y recupera únicamente 5 de ellas

select_movies_query = """
    SELECT CONCAT(title, " (", release_year, ")"),
          collection_in_mil
    FROM movies
    ORDER BY collection_in_mil DESC
"""
with connection.cursor() as cursor:
    cursor.execute(select_movies_query)
    for movie in cursor.fetchmany(size=5):
        print(movie)
    cursor.fetchall()

('Avengers: Endgame (2019)', Decimal('858.8'))
('Avengers: Endgame (2019)', Decimal('858.8'))
('Titanic (1997)', Decimal('659.2'))
('Titanic (1997)', Decimal('659.2'))
('The Dark Knight (2008)', Decimal('535.4'))


**Ejercicio:** Consulte peliculas cuyo año de lanzamiento este por debajo de 2000 y ordene los resultados por recaudo en taquilla de mayor a menor valor.